In [16]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from shapely import wkb
from shapely import errors
import itertools
import matplotlib.pyplot as plt
import datetime as dt
import holidays
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [17]:
df = pd.read_parquet("/Users/michaelbrady/Documents/repo/DU-VIRT-AI-PT-11-2024-U-LOLC/ny_ride_fair_optimization/nik_folder/Resources/model_df.parquet")

In [18]:
pd.set_option('display.max_columns', None)
df

,"('second_of_day',)","('day_of_year',)","('weekend',)","('holiday',)","('morning_rush',)","('evening rush',)","('prcp',)","('temp',)","('PUx',)","('PUy',)","('DOx',)","('DOy',)","('distance',)","('duration(sec)',)","('fare',)","('tolls',)","('airport',)","('congestion',)","('total',)","('class',)","('PU_Bronx',)","('PU_Brooklyn',)","('PU_Manhattan',)","('PU_Queens',)","('PU_Staten Island',)","('DO_Bronx',)","('DO_Brooklyn',)","('DO_Manhattan',)","('DO_Queens',)","('DO_Staten Island',)"
0,5,1,False,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,-8.232220e+06,4.983524e+06,18.85,1599.0,70.00,6.94,0.0,0.0,95.88,0,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,-8.218525e+06,4.993771e+06,2.47,785.0,14.59,0.00,0.0,0.0,14.59,1,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,-8.232025e+06,4.972525e+06,2.02,474.0,20.62,0.00,0.0,0.0,20.62,2,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,-8.216190e+06,4.976140e+06,2.47,459.0,23.20,0.00,0.0,0.0,23.20,2,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,-8.235160e+06,4.976716e+06,1.06,335.0,7.90,0.00,0.0,2.5,15.48,0,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,False,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,-8.221850e+06,4.985513e+06,2.48,534.0,14.38,0.00,0.0,0.0,14.38,2,True,False,False,False,False,True,False,False,False,False
6044443,85864,366,False,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,-8.231941e+06,4.963280e+06,1.58,488.0,15.01,0.00,0.0,0.0,15.01,2,False,True,False,False,False,False,True,False,False,False
6044444,85870,366,False,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,-8.236077e+06,4.972248e+06,0.60,256.0,5.80,0.00,0.0,2.5,10.80,0,False,False,True,False,False,False,False,True,False,False
6044445,86042,366,False,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,-8.224081e+06,4.965819e+06,1.36,266.0,8.92,0.00,0.0,0.0,8.92,1,False,False,False,True,False,False,True,False,False,False


In [19]:
pd.set_option('display.max_columns', None)
df.columns = df.columns.str.replace(r"[()',]", "", regex=True).str.strip()
df = df.replace({True: 1, False: 0})

df.rename(columns={"fore": "fare"}, inplace=True)
df.rename(columns={"class": "service"}, inplace=True)
df

/var/folders/vt/jy8dcmxj29vfz95p7lls9h1m0000gn/T/ipykernel_88350/2756985136.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,DOx,DOy,distance,durationsec,fare,tolls,airport,congestion,total,service,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
0,5,1,0,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,-8.232220e+06,4.983524e+06,18.85,1599.0,70.00,6.94,0.0,0.0,95.88,0,0,0,0,1,0,0,0,1,0,0
1,9,1,0,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,-8.218525e+06,4.993771e+06,2.47,785.0,14.59,0.00,0.0,0.0,14.59,1,1,0,0,0,0,1,0,0,0,0
2,33,1,0,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,-8.232025e+06,4.972525e+06,2.02,474.0,20.62,0.00,0.0,0.0,20.62,2,0,0,0,1,0,0,1,0,0,0
3,38,1,0,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,-8.216190e+06,4.976140e+06,2.47,459.0,23.20,0.00,0.0,0.0,23.20,2,0,0,0,1,0,0,0,0,1,0
4,42,1,0,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,-8.235160e+06,4.976716e+06,1.06,335.0,7.90,0.00,0.0,2.5,15.48,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,0,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,-8.221850e+06,4.985513e+06,2.48,534.0,14.38,0.00,0.0,0.0,14.38,2,1,0,0,0,0,1,0,0,0,0
6044443,85864,366,0,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,-8.231941e+06,4.963280e+06,1.58,488.0,15.01,0.00,0.0,0.0,15.01,2,0,1,0,0,0,0,1,0,0,0
6044444,85870,366,0,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,-8.236077e+06,4.972248e+06,0.60,256.0,5.80,0.00,0.0,2.5,10.80,0,0,0,1,0,0,0,0,1,0,0
6044445,86042,366,0,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,-8.224081e+06,4.965819e+06,1.36,266.0,8.92,0.00,0.0,0.0,8.92,1,0,0,0,1,0,0,1,0,0,0


In [20]:
df["service"].value_counts()

service
2    2074749
1    2019704
0    1949994
Name: count, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [22]:
df_taxi = df[(df["service"] == 0) & (df["durationsec"] > 0)]
df_uber = df[(df["service"] == 1) & (df["durationsec"] > 0)]
df_lyft = df[(df["service"] == 2) & (df["durationsec"] > 0)]

# removing total and fare and for leakage

features = ["second_of_day", "day_of_year", "PUx", "PUy", "DOx", "DOy", "distance", "morning_rush", "evening rush",
            "prcp", "temp", "holiday", "weekend", "airport", "congestion", "PU_Bronx", "PU_Brooklyn", "durationsec",
            "PU_Manhattan", "PU_Queens", "PU_Staten Island", "DO_Bronx", "DO_Brooklyn", 
            "DO_Manhattan", "DO_Queens", "DO_Staten Island"]

In [23]:
"""
taxi_model = LinearRegression().fit(df_taxi[features], df_taxi["fare"])
uber_model = LinearRegression().fit(df_uber[features], df_uber["fare"])
lyft_model = LinearRegression().fit(df_lyft[features], df_lyft["fare"])
"""

'\ntaxi_model = LinearRegression().fit(df_taxi[features], df_taxi["fare"])\nuber_model = LinearRegression().fit(df_uber[features], df_uber["fare"])\nlyft_model = LinearRegression().fit(df_lyft[features], df_lyft["fare"])\n'

In [24]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# final NaN check
df = df.dropna()

y = df_taxi["durationsec"] 
X = df_taxi[features]

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV


In [45]:

param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
}

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.best_estimator_.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb:.2f}")
r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)

Best Parameters: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.6}
XGBoost MAE: 14.85
R² via r2_score: 0.988426559720595


In [ ]:
df_taxi["durationsec"].describe()

count    1.949613e+06
mean     1.046743e+03
std      2.170717e+03
min     -8.248000e+04
25%      4.600000e+02
50%      7.630000e+02
75%      1.256000e+03
max      5.673240e+05
Name: durationsec, dtype: float64

In [46]:
y = df_uber["durationsec"] 
X = df_uber[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.best_estimator_.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb}")
r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)



Best Parameters: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1.0}
XGBoost MAE: 9.250169313382763
R² via r2_score: 0.9946710198085753


In [ ]:
y = df_lyft["durationsec"] 
X = df_lyft[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)


Best Parameters: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.6}


AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_predict'

In [43]:

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.best_estimator_.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb}")

r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)


Best Parameters: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.6}
XGBoost MAE: 14.851331980185737
R² via r2_score: 0.988426559720595


In [52]:
xgb_model = random_search
i = 1900002
test_row = df_uber.iloc[i]
test_df = test_row[features].to_frame().T

predicted_value = xgb_model.predict(test_df)
print(f"Predicted Value: {predicted_value[0]:.2f}")
print(f"Actual Value {test_row["durationsec"]}")


Predicted Value: 614.39
Actual Value 620.0
